In [1]:
import pandas as pd

In [2]:
excel_file_path = 'C:/Users/ASUS/OneDrive/Documents/實習/Rebas/CPBL_2023.xlsx'  # 請替換成你的 Excel 檔案路徑
sheet_name1 = 'AWAY-打席資訊'  # 請替換成你要載入的工作表名稱
sheet_name2 = 'HOME-打席資訊'

In [3]:
away = pd.read_excel(excel_file_path, sheet_name=sheet_name1)
home = pd.read_excel(excel_file_path, sheet_name=sheet_name2)

In [4]:
cols = ['客場','主場','打席結果','落點1','落點2','彈道']

In [5]:
away_play = away[cols]
home_play = home[cols]

## 客隊投手

In [6]:
def calculate_so_go_count(series):
    so_count = (series == 'SO').sum()
    ubb_count = (series == 'uBB').sum()
    hbp_count = (series == 'HBP').sum()
    hr_count = (series == 'HR').sum()
    return pd.Series({'SO': so_count,
                      'HR':hr_count
                     , 'uBB':ubb_count
                     , 'HBP':hbp_count})

In [7]:
resault_home=home_play .groupby(['客場'])['打席結果'].apply(calculate_so_go_count).reset_index()
resault_home = resault_home.rename(columns={'level_1': '打席結果', '打席結果': '次數'})
resault_home

,客場,打席結果,次數
0,中信兄弟,SO,430
1,中信兄弟,HR,31
2,中信兄弟,uBB,163
3,中信兄弟,HBP,35
4,味全龍,SO,381
5,味全龍,HR,20
6,味全龍,uBB,174
7,味全龍,HBP,39
8,富邦悍將,SO,340
9,富邦悍將,HR,30


In [8]:
resault_home['次數'] = resault_home['次數'].astype(int)

final_resault_home = pd.pivot_table(resault_home, index=['客場'], columns='打席結果', values='次數', aggfunc='sum', fill_value=0)

final_resault_home

打席結果,HBP,HR,SO,uBB
客場,,,,
中信兄弟,35,31,430,163
味全龍,39,20,381,174
富邦悍將,33,30,340,201
樂天桃猿,34,35,374,196
統一7-ELEVEn獅,29,32,340,182


In [9]:
def classify_fly_ball(row):
    #if row['彈道'] == 'F' and row['落點1'] in ['7.0','8.0','9.0']:
        #return '外野飛球'
    if row['彈道'] == 'P' and row['落點1'] not in ['7.0','8.0','9.0']:
        return '內野飛球'
    #elif row['彈道'] == 'P' and row['落點1'] in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        #return '內野飛球'
    elif row['彈道'] == 'F' and row['落點1'] not in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        return '外野飛球'
    elif row['彈道'] == 'L':
        return '平飛球'
    elif row['彈道'] == 'G':
        return '滾地球'
    else:
        return '其他'

home_play['飛球分類'] = home_play.apply(classify_fly_ball, axis=1)
away_play['飛球分類'] = away_play.apply(classify_fly_ball, axis=1)
# 打印结果


C:\Users\ASUS\AppData\Local\Temp\ipykernel_18392\1790990382.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_play['飛球分類'] = home_play.apply(classify_fly_ball, axis=1)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_18392\1790990382.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_play['飛球分類'] = away_play.apply(classify_fly_ball, axis=1)


In [10]:
def calculate_so_go_count2(series):
    iff_count = (series == '內野飛球').sum()
    off_count = (series == '外野飛球').sum()
    ld_count = (series == '平飛球').sum()
    gr_count = (series == '滾地球').sum()
    return pd.Series({'內野飛球': iff_count,
                      '外野飛球':off_count
                     , '平飛球':ld_count
                     , '滾地球':gr_count})

In [11]:
resault_home2=home_play.groupby(['客場'])['飛球分類'].apply(calculate_so_go_count2).reset_index()
resault_home2 = resault_home2.rename(columns={'level_1': '擊球分類', '飛球分類': '次數'})

resault_home2['次數'] = resault_home2['次數'].astype(int)

final_resault_home2 = pd.pivot_table(resault_home2, index=['客場'], columns='擊球分類', values='次數', aggfunc='sum', fill_value=0)

final_resault_home2

擊球分類,內野飛球,外野飛球,平飛球,滾地球
客場,,,,
中信兄弟,119,531,193,749
味全龍,98,564,233,732
富邦悍將,93,511,221,834
樂天桃猿,125,519,247,858
統一7-ELEVEn獅,124,603,208,698


In [12]:
pitcher_home = pd.merge(final_resault_home, final_resault_home2, on='客場', how='inner')

In [13]:
constant=3.4377164595959897
pitcher_home['tRA']=((pitcher_home ['uBB']*0.354188
                 +pitcher_home ['HBP']*0.372947
                 -pitcher_home ['SO']*0.304666
                 +pitcher_home ['HR']*1.711948
                 -pitcher_home ['滾地球']*0.077254
                 -pitcher_home ['內野飛球']*0.300934
                 +pitcher_home ['外野飛球']*0.048510
                 +pitcher_home ['平飛球']*0.307350)/
                (pitcher_home ['SO']
                 +pitcher_home ['滾地球']*0.709957
                +pitcher_home ['平飛球']*0.290352
                +pitcher_home ['內野飛球']*0.987002
                +pitcher_home ['外野飛球']*0.711651)*27)+constant

In [14]:
pitcher_home

,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA
客場,,,,,,,,,
中信兄弟,35,31,430,163,119,531,193,749,3.317683
味全龍,39,20,381,174,98,564,233,732,3.318558
富邦悍將,33,30,340,201,93,511,221,834,4.013331
樂天桃猿,34,35,374,196,125,519,247,858,4.184910
統一7-ELEVEn獅,29,32,340,182,124,603,208,698,3.574928


## 主隊投手

In [15]:
#groupby(['主場']) 切換算主客場
resault_away=away_play .groupby(['主場'])['打席結果'].apply(calculate_so_go_count).reset_index()
resault_away = resault_away.rename(columns={'level_1': '打席結果', '打席結果': '次數'})

resault_away['次數'] = resault_away['次數'].astype(int)

final_resault_away = pd.pivot_table(resault_away, index=['主場'], columns='打席結果', values='次數', aggfunc='sum', fill_value=0)

final_resault_away

打席結果,HBP,HR,SO,uBB
主場,,,,
中信兄弟,26,44,422,171
味全龍,32,38,389,187
富邦悍將,34,22,398,175
樂天桃猿,35,36,375,185
統一7-ELEVEn獅,24,41,417,186


In [16]:
##groupby(['主場']) 切換算主客場
resault_away2=away_play.groupby(['主場'])['飛球分類'].apply(calculate_so_go_count2).reset_index()
resault_away2 = resault_away2.rename(columns={'level_1': '擊球分類', '飛球分類': '次數'})

resault_away2['次數'] = resault_away2['次數'].astype(int)

final_resault_away2 = pd.pivot_table(resault_away2, index=['主場'], columns='擊球分類', values='次數', aggfunc='sum', fill_value=0)

final_resault_away2

擊球分類,內野飛球,外野飛球,平飛球,滾地球
主場,,,,
中信兄弟,123,558,244,794
味全龍,84,574,279,741
富邦悍將,112,530,229,855
樂天桃猿,117,631,206,884
統一7-ELEVEn獅,159,618,158,709


In [17]:
pitcher_away = pd.merge(final_resault_away, final_resault_away2, on='主場', how='inner')

In [18]:
constant=3.4377164595959897
pitcher_away ['tRA']=((pitcher_away ['uBB']*0.354188
                 +pitcher_away ['HBP']*0.372947
                 -pitcher_away ['SO']*0.304666
                 +pitcher_away ['HR']*1.711948
                 -pitcher_away['滾地球']*0.077254
                 -pitcher_away ['內野飛球']*0.300934
                 +pitcher_away ['外野飛球']*0.048510
                 +pitcher_away ['平飛球']*0.307350)/
                (pitcher_away ['SO']
                 +pitcher_away ['滾地球']*0.709957
                +pitcher_away ['平飛球']*0.290352
                +pitcher_away ['內野飛球']*0.987002
                +pitcher_away ['外野飛球']*0.711651)*27)+constant

In [19]:
pitcher_away 

,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA
主場,,,,,,,,,
中信兄弟,26,44,422,171,123,558,244,794,3.921732
味全龍,32,38,389,187,84,574,279,741,4.080622
富邦悍將,34,22,398,175,112,530,229,855,3.429491
樂天桃猿,35,36,375,185,117,631,206,884,3.461657
統一7-ELEVEn獅,24,41,417,186,159,618,158,709,3.275303


In [20]:
#pitcher_away.to_excel('客場球隊投手各項目總數.xlsx', index=True)

In [21]:
def calculate_so_go_count3(series):
    so_count = (series == 'SO').sum()
    go_count = (series == 'GO').sum()
    gidp_count = (series == 'GIDP').sum()
    fo_count = (series == 'FO').sum()
    sh_count = (series == 'SH').sum()
    sf_count = (series == 'SF').sum()
    hit_count = (series == '1B').sum()
    hit2_count = (series == '2B').sum()
    hit3_count = (series == '3B').sum()
    hit4_count = (series == 'HR').sum()
    ubb_count = (series == 'uBB').sum()
    e_count = (series == 'E').sum()
    fc_count = (series == 'FC').sum()
    hbp_count = (series == 'HBP').sum()
    she_count = (series == 'SH_E').sum()
    shfc_count = (series == 'SH_FC').sum()
    return pd.Series({'SO': so_count, 'GO': go_count ,'GIDP':gidp_count, 'FO':fo_count, 'SH':sh_count, 'SF':sf_count, '1B':hit_count
                     , '2B':hit2_count
                     , '3B':hit3_count
                     , 'HR':hit4_count
                     , 'uBB':ubb_count
                     , 'E':e_count
                     , 'FC':fc_count
                     , 'HBP':hbp_count
                     , 'SH_E':she_count
                     , 'SH_FC':shfc_count})

In [22]:
#groupby(['主場']) 切換算主客場
away_count=away_play.groupby(['主場'])['打席結果'].apply(calculate_so_go_count3).reset_index()
away_count =away_count.rename(columns={'level_1': '打席結果', '打席結果': '次數'})

away_count ['次數'] =away_count ['次數'].astype(int)

final_count_away = pd.pivot_table(away_count , index=['主場'], columns='打席結果', values='次數', aggfunc='sum', fill_value=0)

final_count_away

打席結果,1B,2B,3B,E,FC,FO,GIDP,GO,HBP,HR,SF,SH,SH_E,SH_FC,SO,uBB
主場,,,,,,,,,,,,,,,,
中信兄弟,405,88,8,10,65,561,55,450,26,44,20,30,1,0,422,171
味全龍,372,95,12,16,62,581,53,415,32,38,14,26,0,2,389,187
富邦悍將,416,79,16,27,83,541,39,456,34,22,17,30,3,0,398,175
樂天桃猿,495,69,9,36,99,577,50,423,35,36,17,29,2,2,375,185
統一7-ELEVEn獅,356,75,5,20,49,620,44,381,24,41,20,32,1,1,417,186
